In [20]:
# -*- coding: utf-8 -*-
####################################################
### 上下動算出 from阪下さんExcel
####################################################
import sys
import numpy as np
import pandas as pd
import sip
import line_profiler as lp

# N=255 LPF
lpf=[0.000198929,0.000202058,0.000205152,0.0002082,0.000211175,0.000214037,0.000216727,0.000219175,0.000221291,0.000222975,0.000224109,0.000224563,0.000224194,0.000222846,0.000220355,0.000216546,0.000211237,0.00020424,0.000195363,0.000184413,0.000171195,0.000155519,0.000137197,0.00011605,9.19079E-05,6.4612E-05,3.40184E-05,3.66184E-19,-3.75509E-05,-7.87203E-05,-0.00012357,-0.000172135,-0.000224422,-0.000280407,-0.000340033,-0.000403208,-0.000469806,-0.00053966,-0.000612568,-0.000688286,-0.00076653,-0.000846976,-0.000929256,-0.001012963,-0.001097651,-0.00118283,-0.001267972,-0.001352511,-0.001435846,-0.001517339,-0.001596319,-0.001672086,-0.001743911,-0.001811041,-0.001872701,-0.001928097,-0.001976422,-0.002016856,-0.002048574,-0.002070747,-0.002082549,-0.002083162,-0.002071774,-0.002047595,-0.002009851,-0.001957795,-0.001890711,-0.001807916,-0.001708769,-0.001592671,-0.001459073,-0.001307481,-0.001137454,-0.000948618,-0.000740661,-0.000513339,-0.000266484,-1.41438E-18,0.00028613,0.000591843,0.000916994,0.001261355,0.00162461,0.002006362,0.002406124,0.002823326,0.00325731,0.003707335,0.004172577,0.00465213,0.00514501,0.005650155,0.00616643,0.006692633,0.007227492,0.007769678,0.0083178,0.008870421,0.009426052,0.009983168,0.010540205,0.011095573,0.011647658,0.012194831,0.012735454,0.013267886,0.013790491,0.014301643,0.014799737,0.015283191,0.015750458,0.016200026,0.016630433,0.017040266,0.017428173,0.017792864,0.018133122,0.018447803,0.018735846,0.018996275,0.019228205,0.019430845,0.019603502,0.019745584,0.019856603,0.019936176,0.01998403,0.02,0.01998403,0.019936176,0.019856603,0.019745584,0.019603502,0.019430845,0.019228205,0.018996275,0.018735846,0.018447803,0.018133122,0.017792864,0.017428173,0.017040266,0.016630433,0.016200026,0.015750458,0.015283191,0.014799737,0.014301643,0.013790491,0.013267886,0.012735454,0.012194831,0.011647658,0.011095573,0.010540205,0.009983168,0.009426052,0.008870421,0.0083178,0.007769678,0.007227492,0.006692633,0.00616643,0.005650155,0.00514501,0.00465213,0.004172577,0.003707335,0.00325731,0.002823326,0.002406124,0.002006362,0.00162461,0.001261355,0.000916994,0.000591843,0.00028613,-1.41438E-18,-0.000266484,-0.000513339,-0.000740661,-0.000948618,-0.001137454,-0.001307481,-0.001459073,-0.001592671,-0.001708769,-0.001807916,-0.001890711,-0.001957795,-0.002009851,-0.002047595,-0.002071774,-0.002083162,-0.002082549,-0.002070747,-0.002048574,-0.002016856,-0.001976422,-0.001928097,-0.001872701,-0.001811041,-0.001743911,-0.001672086,-0.001596319,-0.001517339,-0.001435846,-0.001352511,-0.001267972,-0.00118283,-0.001097651,-0.001012963,-0.000929256,-0.000846976,-0.00076653,-0.000688286,-0.000612568,-0.00053966,-0.000469806,-0.000403208,-0.000340033,-0.000280407,-0.000224422,-0.000172135,-0.00012357,-7.87203E-05,-3.75509E-05,3.66184E-19,3.40184E-05,6.4612E-05,9.19079E-05,0.00011605,0.000137197,0.000155519,0.000171195,0.000184413,0.000195363,0.00020424,0.000211237,0.000216546,0.000220355,0.000222846,0.000224194,0.000224563,0.000224109,0.000222975,0.000221291,0.000219175,0.000216727,0.000214037,0.000211175,0.0002082,0.000205152,0.000202058,0.000198929]

## 前後[フィルタ係数のN数]/2点のFIR
def fir(x, b):
    """FIRフィルタ
    x: 入力信号
    b: フィルタ係数"""
    y = [0.0] * len(x)  # フィルタの出力信号
    N = len(b) - 1      # フィルタ係数の数
    for n in range(len(x)):
        if n>N/2.0-1:
            try:
                y[n] = np.nansum(x[n-N/2.0:n+N/2.0+1]*b)
            except:pass
        else:y[n]=np.nan
    #y[:(N/2)]=[np.nan]*(N/2) # FIR点数未達の空データ削除
    return y

## 上下動算出の加速度積分処理
def CalcPitch(x,t,g,gg):
    p = [np.nan] * len(x)
    for i in range(len(x)):
        if x[i]==x[i]: # and x[i-1]==x[i-1]: # is not NaN
            if p[i-1]!=p[i-1] or x[i-1]!=x[i-1]:
                p[i] = (x[i]) / 2.0 / g * gg / 2.0 * t**2.0
            else:
                p[i] = (x[i] + x[i-1]) / 2. / g * gg / 2. * t**2.0 + p[i-1]
    return p

## FindPeaks 波形の山谷を検出
def findPeaks(tmp,point) :
    pk = np.zeros(len(tmp))
    vl = np.zeros(len(tmp))
    for i in range(len(tmp)) :
        if i > point :
            chkp = max(tmp[i-point:i+point])
            chkv = min(tmp[i-point:i+point])
            if tmp[i] == chkp and tmp[i] != tmp[i-1] and tmp[i] != tmp[i-2] and tmp[i] != tmp[i-3] and tmp[i] != tmp[i-4]:
                pk[i] = 1
            if tmp[i] == chkv and tmp[i] != tmp[i-1] and tmp[i] != tmp[i-2] and tmp[i] != tmp[i-3] and tmp[i] != tmp[i-4]:
                vl[i] = 1
    return pk,vl

## ファイル選択ダイアログ
def FileSelectDialog(path='',multi=False):
    import tkinter
    from tkinter import filedialog
    from tkinter import messagebox

    root = tkinter.Tk()
    root.withdraw()

    ftype=[('Csv file','*.csv'),('All files','*.*')]
    ftitle="処理するデータファイルを選択して下さい"
    defext='.csv'

    if len(path) != 0:
        idir = path.replace("\\","/")
    else:
        idir='c:/'
    filename = filedialog.askopenfilename(initialdir=idir,defaultextension=defext,title=ftitle,multiple=multi,filetypes=ftype)

    if filename:
        #tkMessageBox.showinfo('選択したファイル', filename)
        fpath = filename
    else:
        fpath = ""

    return fpath #.replace("u'","").replace("'","")

def main():
    global lpf

    print("========= Start Process. =========")

    # ファイル読み込み
    fname=FileSelectDialog(path="c:\\temp\\data_python\\test\\")
    print('SOURCE ||'+fname)
    try:
        # ファイル生成元をチェックしてデータをセット
        df=pd.read_csv(fname,delimiter=",",comment="/")
        if df.shape[1] == 3: # for ACP
            df = pd.read_csv(fname, delimiter=',',skiprows=5, header=0,parse_dates=[2], encoding="shift-jis")
            accX=np.array(df[u'ACC_X'])
            accY=np.array(df[u'ACC_Y'])
            accZ=np.array(df[u'ACC_Z'])
            cnt=np.array(df[u'NUM'])
        elif df.shape[1]==17: # for OS Logger
            df = pd.read_csv(fname, delimiter=',',skiprows=None, header=0,parse_dates=[1], encoding="shift-jis")
            accX=np.array(df[u'accX'])
            accY=np.array(df[u'accY'])
            accZ=np.array(df[u'accZ'])
            pitch=np.array(df[u'pitch'])
            cnt=np.array(df[u'count'])
        elif df.shape[1]==18: # for Android Logger
            df = pd.read_csv(fname, delimiter=',',skiprows=None, header=0,parse_dates=[1], encoding="shift-jis")
            accX=np.array(df[u'accX'])
            accY=np.array(df[u'accY'])
            accZ=np.array(df[u'accZ'])
            pitch=np.array(df[u'pitch'])
            cnt=np.array(df[u'Count'])
        elif df.shape[1]==24: # for iOS ODE-Logger NewUpDown
            df = pd.read_csv(fname, delimiter=',',skiprows=None, header=0,parse_dates=[1], encoding="shift-jis")
            accX=np.array(df[u'accX'])
            accY=np.array(df[u'accY'])
            accZ=np.array(df[u'accZ'])
            pitch=np.array(df[u'pitch'])
            cnt=np.array(df[u'id'])
        else:
            df = pd.read_csv(fname, delimiter=',',header=0,)
            accX=np.array(df[u'accX'])
            accY=np.array(df[u'accY'])
            accZ=np.array(df[u'accZ'])
            pitch=np.array(df[u'pitch'])
            cnt=np.array(df[u'count'])
        d=np.array(df)
    except:
        print("!!!!! Process Failuer by FileOpen.")
        sys.exit()

    # 前傾補正-静止系として
    try:
        accZf = np.array([z/np.cos(np.radians(p)) for z,p in zip(accZ,pitch)]) #accZ/cospicth
    except:
        accZf = accZ

    # 上下動算出 from Excel
    # 1.元データを前後255点のLPF処理
    x_lpf=fir(accX,lpf)
    y_lpf=fir(accY,lpf)
    z_lpf=fir(accZ,lpf)
    zf_lpf=fir(accZf,lpf)
    # 2.元データからLPF後のデータを引く
    x_hpf = accX - x_lpf
    y_hpf = accY - y_lpf
    z_hpf = accZ - z_lpf
    zf_hpf= accZf - zf_lpf
    # 3.1のLPF後データから姿勢角を算出
    psi = [np.arctan(x/np.sqrt(y**2+z**2))*180.0/np.pi for x,y,z in zip(x_lpf,y_lpf,z_lpf)]
    theta = [np.arctan(y/np.sqrt(x**2+z**2))*180.0/np.pi for x,y,z in zip(x_lpf,y_lpf,z_lpf)]
    phai = [np.arctan(np.sqrt(x**2+y**2)/z) for x,y,z in zip(x_lpf,y_lpf,z_lpf)]
    psif = [np.arctan(x/np.sqrt(y**2+z**2))*180.0/np.pi for x,y,z in zip(x_lpf,y_lpf,zf_lpf)]
    thetaf = [np.arctan(y/np.sqrt(x**2+z**2))*180.0/np.pi for x,y,z in zip(x_lpf,y_lpf,zf_lpf)]
    phaif = [np.arctan(np.sqrt(x**2+y**2)/z) for x,y,z in zip(x_lpf,y_lpf,zf_lpf)]
    # 4.地上座標でのZ軸方向の加速度を算出
    accZ2 = [1.0/np.cos(p/180*np.pi)*z for p,z in zip(phai,z_hpf)]
    accZf2 = [1.0/np.cos(p/180*np.pi)*z for p,z in zip(phaif,zf_hpf)]
    # 5.4の加速度を積分
    Vm = CalcPitch(accZ2,0.05,16,9.8)  # t=1/20Hz,1G=16,g=9.8m/s^2
    Vmf = CalcPitch(accZf2,0.05,16,9.8)  # t=1/20Hz,1G=16,g=9.8m/s^2

    # 山谷の集計
    pk, vl = findPeaks(Vm,5)
    diff = [np.nan] * len(df) #np.zeros(len(df))
    p1,p2=0,0
    for i in range(len(df)):
        if pk[i]==1:
            p1 = i
        if vl[i]==1 and p1!=0:
            diff[i]=Vm[p1]-Vm[i]
            p1,p2=0,i
    print("Diff Max:%f,Min:%f,Ave:%f,Std:%f" % (np.nanmax(diff),np.nanmin(diff),np.nanmean(diff),np.std([x for x in diff if x==x])))

    # ファイル出力: 元データの後ろに結合
    out=pd.DataFrame(np.c_[d,x_lpf,y_lpf,z_lpf,x_hpf,y_hpf,z_hpf,psi,theta,phai,
                    accZ2,Vm,pk,vl,diff,accZf,zf_lpf,zf_hpf,phaif,accZf2,Vmf])
    try:
        out.columns=[np.r_[df.columns,['x-lpf','y-lpf','z-lpf','x-hpf','y-hpf','z-hpf',
                                    'psi','theta','phai','accZ-2','VerticalMotion',
                                    'Vm-PK','Vm-VL','Vm-diff','accZf','zf_lpf',
                                    'zf_hpf','phaif','accZf2','Vmf']]]
    except:
        print("!!!!! Process Failuer by Column Name Add.")

    fname_out = fname.replace(".csv","_pyVmv.csv")
    out.to_csv(fname_out,index=None,encoding='shift-jis')
    print("Output || "+fname_out)
    print("========= End Process. =========")

if __name__ == '__main__':
    main()


========= Start Process. =========
SOURCE ||/Users/jins/Downloads/2016414-122517_E6110CB3-224C-BA47-1581-61A911E40FCB.csv


/Users/jins/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:24: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Diff Max:0.089881,Min:0.000087,Ave:0.015576,Std:0.019633
Output || /Users/jins/Downloads/2016414-122517_E6110CB3-224C-BA47-1581-61A911E40FCB_pyVmv.csv
========= End Process. =========
